In [1]:
import pandas as pd 
from scipy.stats import fisher_exact

In [2]:
#uploading the files we need

#nonhaplotype sites
nonhaplotype_sites_file = "../files/nonhaplotype_site_allele_counts.txt"
nonhaplotype_sites = pd.read_csv(nonhaplotype_sites_file, sep = "\t")

#haplotype sites w nonreversion alleles
haplotype_sites_nonreversions_file = "../files/haplotype_site_nonreversion_allele_counts.txt"
haplotype_sites_nonreversions = pd.read_csv(haplotype_sites_nonreversions_file, sep = "\t")

#haplotype sites w reversion alleles
#note we are only looking at SNVs -- NZB will have 89 vars
haplotype_sites_reversions_file = "../files/haplotype_site_reversion_allele_counts_w_NUMT_correction.txt"
haplotype_sites_reversions = pd.read_csv(haplotype_sites_reversions_file, sep = "\t")

In [3]:
#create a dictionary that contains all haplotype info {strain:[[site, ref, alt], [site, ref, alt] ... ], strain ...}

#create a dataframe with the haplotype information

#gathering the reversion site info
haplotype_site_rev_info = haplotype_sites_reversions.drop_duplicates(subset = ["STRAIN", "START"], \
                                                                 keep = "first")[["STRAIN", \
                                                                                  "START", "REV_B6_ALLELE",\
                                                                                 "ORG_CONPLASTIC_ALLELE"]]

#gathering the strain names
strains = list(haplotype_site_rev_info["STRAIN"].unique())

#creating the dictionary we will iterate through in the loop
haplotype_rev_dict = {}
for strain in strains:
    #take the position, ref, and alt alleles for all haploptypes for a given strain & place into one list of lists 
    haplotypes = haplotype_site_rev_info[haplotype_site_rev_info["STRAIN"] == strain]
    haplotypes_lst = haplotypes[["START", "REV_B6_ALLELE", "ORG_CONPLASTIC_ALLELE"]].values.tolist()
    #add the list of lists as a value to the strain keys
    haplotype_rev_dict[strain] = haplotypes_lst

In [4]:
#in the for-loop sequence we calculate the components of the fisher's exact test 
#we will conduct the fisher's exact test in R because scipy's fisher's exact test calculates the unconditional MLE 
#of the OR, which does not follow the theory (in other words R's fisher's exact OR is more accurate)

results = []

for age in ["OLD", "YOUNG"]:
    #strains is the list we created before
    for strain in strains:
        #set the list of haplotype site we need to visit for the given strain
        print(strain)
        #these positions will change when we look at a new strain 
        positions = haplotype_rev_dict[strain]
        
        for tissue in ["Brain", "Heart", "Liver"]:
            #subset our data so we can iterate through a smaller df -- the subset will only keep information for the 
            #condition we're interested in looking at strain x tissue x age 
            
            nonhaplotype_sites_subset = nonhaplotype_sites[(nonhaplotype_sites["AGE_BIN"] == age) & \
                                                          (nonhaplotype_sites["STRAIN"] == strain) & \
                                                          (nonhaplotype_sites["TISSUE"] == tissue)]
            
            haplotype_sites_nonreversions_subset = haplotype_sites_nonreversions[(haplotype_sites_nonreversions["AGE_BIN"] == age) & \
                                                          (haplotype_sites_nonreversions["STRAIN"] == strain) & \
                                                          (haplotype_sites_nonreversions["TISSUE"] == tissue)]
            
            haplotype_sites_reversions_subset = haplotype_sites_reversions[(haplotype_sites_reversions["AGE_BIN"] == age) & \
                                                          (haplotype_sites_reversions["STRAIN"] == strain) & \
                                                          (haplotype_sites_reversions["TISSUE"] == tissue)]
            
            #this for loop is designed to create the df we need for our fisher's exact test
            #we loop through the list of lists in positions = [[site, ref, alt], [site, ref, alt], ...]
            for index in range(len(positions)): 
                #set the information for the haplotype site
                pos = positions[index][0]
                rev_B6_allele = positions[index][1]
                org_conplastic_allele = positions[index][2]
                
                #initialize inputs for the contingency table
                rev_mut_at_haplotype_site = 0
                other_mut_at_haplotype_site = 0
                rev_mut_at_background_sites = 0
                other_muts_at_background_sites = 0
                
                #subsetting the haplotype_reversion data to extract info for the positions
                #dim of haplotype_reversion_data_by_site = 1
                haplotype_reversion_data_by_site = haplotype_sites_reversions_subset[haplotype_sites_reversions_subset["START"] \
                                                                                     == pos]
                
                if len(haplotype_reversion_data_by_site) != 1:
                    print("Error: haplotype reversion info size != 1")
                    
                #calculating the number of rev_B6_alleles at the haplotype site 
                rev_mut_at_haplotype_site += haplotype_reversion_data_by_site["REVERSION_ALLELE_DEPTH"].values[0]
                
                #now we calculate the other mutations at the haplotype site
                #Ex: G>A is the rev mut, other muts at site are G>C and G>T -- we do not include the case where there
                #is no mutation because we condition on the fact that a mutation has occurred 
               
                other_mut_at_haplotype_site += haplotype_sites_nonreversions_subset[(haplotype_sites_nonreversions_subset["START"] == pos)]["COND_ALLELE_COUNT"].sum()
                
                #we calculate the background position entries: rev_mut_at_background_sites & other_muts_at_background_sites
                #before making these calculations we need to make a key note -- the reversion information needs to be recoded 
                
                if org_conplastic_allele == "C" or org_conplastic_allele == "A":
                    #print("original mut:", org_conplastic_allele, rev_B6_allele)
                    #in our nonhaplotype sites we have recoded the mutations to avoid double counting such that C>G and A>T
                    recoded_ref = {"C": "G", "A" : "T"}
                    #we need to also recode our alt allele so that we have the complimentary mutation overall
                    recoded_alt = {"T": "A", "G":"C", "A": "T", "C": "G"}
                    org_conplastic_allele, rev_B6_allele = recoded_ref[org_conplastic_allele],  recoded_alt[rev_B6_allele]
                    #print("recoded ouput:", org_conplastic_allele, rev_B6_allele)
                    
                    
                #now gathering the count of rev_mut_at_background_sites 
                rev_mut_at_background_sites_df = nonhaplotype_sites_subset[(nonhaplotype_sites_subset["RECODED_REF"] \
                                                                            == org_conplastic_allele) & (nonhaplotype_sites_subset["RECODED_ALT"] \
                                                                            == rev_B6_allele)]

                #sum of all the reversion mutation type occuring at other sites
                rev_mut_at_background_sites += rev_mut_at_background_sites_df["COND_ALLELE_COUNT"].values.sum()
                
                #finally calculating other_muts_at_background_sites
                #we want 1) ref allele == org_conplastic_strain and alt allele != rev_B6_allele 
                #no mutation has cond_allele_count = 0 
                #sum the cond_allele_count 
                
                other_mut_at_background_sites_df = nonhaplotype_sites_subset[(nonhaplotype_sites_subset["RECODED_REF"] \
                                                                            == org_conplastic_allele) & (nonhaplotype_sites_subset["RECODED_ALT"] \
                                                                            != rev_B6_allele)]
                
                other_muts_at_background_sites += other_mut_at_background_sites_df["COND_ALLELE_COUNT"].values.sum()
                
                results.append([strain, tissue, age, pos, org_conplastic_allele, rev_B6_allele,\
                                rev_mut_at_haplotype_site, other_mut_at_haplotype_site,\
                                rev_mut_at_background_sites, other_muts_at_background_sites])

output_df = pd.DataFrame(results, columns = ["STRAIN", "TISSUE", "AGE_BIN", "POS", "ORG_CONPLASTIC_ALLELE",\
                                             "REV_B6_ALLELE", "REV_MUT_AT_HAPLOSITE", "OTHER_MUT_AT_HAPLOSITE", \
                                            "REV_MUT_AT_BACKGROUND_SITE", "OTHER_MUT_AT_BACKGROUND_SITE"])
        

AKR
ALR
FVB
NZB
AKR
ALR
FVB
NZB


In [5]:
output_file = "../files/contingency_tables_entries.txt"
output_df.to_csv(output_file, header=True, index=False, sep = "\t")